In [1]:
import openai
import yaml
from src import utils
import time
from pprint import pprint
import os
import argparse
from tqdm import tqdm

In [2]:
# PROMPT = {
#     '3p' : get_3p_prompt,
#     'cot' : get_CoT_prompt,
# }

In [3]:
api = 'openai'
api_key_file = '/userhomes/philhoon/kt-ai-challenge/api-openai.yaml'

In [4]:
# model_name="gpt-3.5-turbo-0301",
model_name = "gpt-3.5-turbo-0613"

max_token=2048
temperature=0.0

In [5]:
input_file = '/userhomes/philhoon/kt-ai-challenge/data/public_mwp_data_v2_preprocess.jsonl'
# output_path = '/userhomes/philhoon/self-debate/result'

In [6]:
# openai key
key_dict = utils.get_keys(api_key_file)
openai.api_key = key_dict[api]

In [7]:
output_file = '/userhomes/philhoon/kt-ai-challenge/data/public_mwp_data_v2_reasoning.jsonl'
print(f'output_file : {output_file}')

output_file : /userhomes/philhoon/kt-ai-challenge/data/public_mwp_data_v2_reasoning.jsonl


In [8]:
# openai-gpt-3.5-turbo-0301-2048-0.0-gsm8k-test.jsonl -> openai-gpt-3.5-turbo-0301-3p-2048-0.0-gsm8k-test.jsonl
# openai-gpt-3.5-turbo-0301-gsm8k-test.jsonl -> openai-gpt-3.5-turbo-0301-3p-inf-1.0-gsm8k-test.jsonl
# openai-gpt-3.5-turbo-0301-2048-0.0-gsm8k-test.jsonl -> openai-gpt-3.5-turbo-0301-3p-2048-0.0-gsm8k-test.jsonl

In [9]:
# output_file = output_path + '/' + 'gsm8k-korean.jsonl'
# print(output_file)

In [10]:
# Jsonlines writier
jsonl_writer = utils.JSONLWriter(output_file)

In [11]:
data = utils.read_jsonlines(input_file)

In [12]:
# cnt = 0
# for ins in data:
# #     pprint(ins)
#     print(ins['question'])
#     print(ins['answer'])
#     print('----')
#     cnt += 1
#     if cnt == 20:
#         break

In [13]:
# def get_message(prompt):
#     """
#     Message for chat api
#     """
#     message = {
#         "role" : "user",
#         "content" : prompt
#         }
#     return [message]

In [14]:
def generate_answer(message, model_name, max_token, temperature):
    """
    Generate response
    """
    try:
        completion = openai.ChatCompletion.create(
            model=model_name,
            messages=message,
            temperature=temperature,
            max_tokens=max_token,
            n=1)
    except:
        print("retrying due to an error......")
        time.sleep(20)
        return generate_answer(message, model_name, max_token, temperature)

    return completion

In [15]:
# def get_trans_msg(context):
#     """
#     Message for chat api
#     """

#     msg = [
#         {
#             "role" : "system",
#             "content" : "Translate given setences into Korean."
#         },
#         {
#             "role" : "user",
#             "content" : context
#         }
#     ]
    
#     return msg

In [16]:
def get_reason_mgs(question, answer):
    """
    Message for chat api
    주어진 질문과 답에 대해, 풀이를 수식으로 300자이내로 서술하시오.
    질문: 지름이 12cm, 높이가 14cm인 원기둥의 전개도를 그렸을 때 전개도의 옆면의 둘레는 몇 cm인지 답을 구하시오. (원주율: 3) 답: 100
    """

    msg = [
        {
            "role" : "system",
            "content" : "주어진 질문과 정답에 대해, 풀이를 수식으로 300자이내로 서술하시오."
        },
        {
            "role" : "user",
            "content" : f"질문: {question} 정답:{answer}"
        }
    ]
    
    return msg

In [17]:
model_name

'gpt-3.5-turbo-0613'

In [18]:
max_token

2048

In [19]:
temperature

0.0

In [20]:
data[0].keys()

dict_keys(['class', 'question', 'answer', 'equation', 'option', 'option_value', 'unknown'])

In [21]:
cnt = 0
for instance in tqdm(data):
    query = instance['question']
    ans = instance['answer']
        
    query_msg = get_reason_mgs(query, ans)
    
    
    query_completion = generate_answer(query_msg, model_name, max_token, temperature)
    q_response = query_completion["choices"][0]["message"]["content"]
    
#     print('query_mgs')
#     print(query_msg)
#     print('q_response')
#     print(q_response)
    
    instance['reason'] = q_response
    print(query_msg)
    pprint(instance)

#     jsonl_writer.write_json_line(instance)
    cnt += 1
    if cnt == 10:
        break

  0%|                                                                                                                                                                                                                                              | 1/3750 [00:05<5:40:19,  5.45s/it]

[{'role': 'system', 'content': '주어진 질문과 정답에 대해, 풀이를 수식으로 300자이내로 서술하시오.'}, {'role': 'user', 'content': '질문: 지영이네 반 친구들이 키가 작은 순서대로 줄을 서있습니다. 지영이는 앞에서부터 6번째에 줄을 섰으며, 지영이네 반은 모두 18명입니다. 키가 큰 사람부터 다시 줄을 세운다면, 지영이는 앞에서부터 몇 번째로 줄을 서게 될까요? 정답:13'}]
{'answer': '13',
 'class': '순서정하기1',
 'equation': '18-6+1',
 'option': 'none',
 'option_value': 'none',
 'question': '지영이네 반 친구들이 키가 작은 순서대로 줄을 서있습니다. 지영이는 앞에서부터 6번째에 줄을 섰으며, 지영이네 반은 '
             '모두 18명입니다. 키가 큰 사람부터 다시 줄을 세운다면, 지영이는 앞에서부터 몇 번째로 줄을 서게 될까요?',
 'reason': '풀이: 지영이는 앞에서부터 6번째에 줄을 섰으므로, 키가 큰 사람부터 다시 줄을 세운다면 지영이는 앞에서부터 13번째로 '
           '줄을 서게 됩니다. 이는 전체 인원 수인 18에서 지영이의 위치인 6을 뺀 값인 12에 1을 더한 값과 같습니다. '
           '따라서 지영이는 앞에서부터 13번째로 줄을 서게 됩니다.',
 'unknown': 'none'}


  0%|▏                                                                                                                                                                                                                                             | 2/3750 [00:12<6:35:23,  6.33s/it]

[{'role': 'system', 'content': '주어진 질문과 정답에 대해, 풀이를 수식으로 300자이내로 서술하시오.'}, {'role': 'user', 'content': '질문: 반짇고리 안에는 여러 가지 모양의 단추가 있습니다. 동그란 단추는 6개가 들어있고, 네모난 단추와 세모난 단추는 각각 3개씩 들어 있습니다. 동그란 단추는 네모난 단추보다 몇 개가 더 많습니까? 정답:3'}]
{'answer': '3',
 'class': '산술연산',
 'equation': '6-3',
 'option': 'none',
 'option_value': 'none',
 'question': '반짇고리 안에는 여러 가지 모양의 단추가 있습니다. 동그란 단추는 6개가 들어있고, 네모난 단추와 세모난 단추는 '
             '각각 3개씩 들어 있습니다. 동그란 단추는 네모난 단추보다 몇 개가 더 많습니까?',
 'reason': '동그란 단추는 6개, 네모난 단추는 3개, 세모난 단추는 3개가 있습니다. 동그란 단추는 네모난 단추보다 3개가 더 '
           '많습니다. 이를 수식으로 나타내면, 동그란 단추의 개수(6개) - 네모난 단추의 개수(3개) = 3개입니다. 따라서 '
           '동그란 단추는 네모난 단추보다 3개가 더 많습니다.',
 'unknown': 'none'}


  0%|▏                                                                                                                                                                                                                                             | 3/3750 [00:16<5:37:32,  5.40s/it]

...

[{'role': 'system', 'content': '주어진 질문과 정답에 대해, 풀이를 수식으로 300자이내로 서술하시오.'}, {'role': 'user', 'content': '질문: 정원이는 한 달 동안 용돈을 군것질 40%, 학용품 30%, 도서 20%인 10000원, 기타 10%인 5000원을 사용했다. 군것질에 쓴 금액을 구하시오. 정답:20000'}]
{'answer': '20000',
 'class': '산술연산',
 'equation': '5000/(10/100)*(40/100)',
 'option': 'none',
 'option_value': 'none',
 'question': '정원이는 한 달 동안 용돈을 군것질 40%, 학용품 30%, 도서 20%인 10000원, 기타 10%인 5000원을 '
             '사용했다. 군것질에 쓴 금액을 구하시오.',
 'reason': '정원이가 군것질에 사용한 금액을 구하기 위해서는 용돈의 40%를 계산해야 한다. 용돈은 10000원이므로, 10000 '
           '* 0.4 = 4000원을 군것질에 사용한 것이다. 따라서 정원이가 군것질에 사용한 금액은 4000원이다.',
 'unknown': 'none'}


  0%|▌                                                                                                                                                                                                                                             | 9/3750 [00:47<6:03:05,  5.82s/it]

[{'role': 'system', 'content': '주어진 질문과 정답에 대해, 풀이를 수식으로 300자이내로 서술하시오.'}, {'role': 'user', 'content': '질문: 은경이는 15일마다, 세찬이는 18일마다 가족 농장에 간다고 합니다. 8월 1일에 두 사람이 함께 가족 농장에 갔다면 다음번에 함께 가족 농장에 가는 날을 구하고, 월과 일을 더해보시오. 정답:40'}]
{'answer': '40',
 'class': '수찾기1',
 'equation': 'LCM(15, 18), (10+30)',
 'option': 'none',
 'option_value': 'none',
 'question': '은경이는 15일마다, 세찬이는 18일마다 가족 농장에 간다고 합니다. 8월 1일에 두 사람이 함께 가족 농장에 '
             '갔다면 다음번에 함께 가족 농장에 가는 날을 구하고, 월과 일을 더해보시오.',
 'reason': '은경이는 15일마다 가족 농장에 간다고 했으므로, 8월 1일로부터 다음번에 함께 가는 날은 15일 후인 8월 '
           '16일이다. 세찬이는 18일마다 가족 농장에 간다고 했으므로, 8월 1일로부터 다음번에 함께 가는 날은 18일 후인 '
           '8월 19일이다. 따라서, 두 사람이 함께 가족 농장에 다시 갈 날은 16일과 19일 중에서 더 늦은 날인 19일이다. '
           '월과 일을 더하면 8월 19일은 8+19=27이므로, 정답은 40이다.',
 'unknown': 'none'}


  0%|▌                                                                                                                                                                                                                                             | 9/3750 [00:51<5:59:49,  5.77s/it]

[{'role': 'system', 'content': '주어진 질문과 정답에 대해, 풀이를 수식으로 300자이내로 서술하시오.'}, {'role': 'user', 'content': '질문: 어제는 1cm 3mm, 오늘은 2cm 4mm의 비가 내렸습니다. 어제와 오늘 내린 비의 양은 모두 몇 mm일까요? 정답:37'}]
{'answer': '37',
 'class': '산술연산',
 'equation': '(1*10+3)+(2*10+4)',
 'option': 'none',
 'option_value': 'none',
 'question': '어제는 1cm 3mm, 오늘은 2cm 4mm의 비가 내렸습니다. 어제와 오늘 내린 비의 양은 모두 몇 mm일까요?',
 'reason': '어제 내린 비의 양은 1cm 3mm이므로, 이를 mm로 변환하면 13mm입니다. 오늘 내린 비의 양은 2cm '
           '4mm이므로, 이를 mm로 변환하면 24mm입니다. 따라서 어제와 오늘 내린 비의 양을 합하면 13mm + 24mm = '
           '37mm입니다.',
 'unknown': 'none'}


In [ ]:
# cnt = 0
# for instance in data:
#     question = instance['question']
#     answer_s = utils.extract_answer(instance['answer'])
#     prompt = get_CoT_prompt(question)
#     prompt = get_prompt(question)
#     print(f'prompt : \n {prompt}')
#     message = get_message(prompt)
    
#     continue
#     completion = generate_answer(message, model_name)
    
# #     completion = openai.ChatCompletion.create(
# #                   model=model_name,
# #                   messages=message,
# #                   temperature=0,Pathways Language Model (PaLM): Scaling to 540 Billion Parameters for Breakthrough Performance
# #                   max_tokens=4096,
# #                   n=1)
    
    
#     response = completion["choices"][0]["message"]["content"]
# #     instance['response'] = response
# #     instance['answer_specific'] = answer_s
# #     jsonl_writer.write_json_line(instance)
#     print(f'respones : \n {response}')
#     cnt += 1
#     if cnt == 10:
#         break

In [ ]:
pprint(instance)